In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import PIL.Image as Image
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#model = create_model()
model = keras.saving.load_model('model2')

In [3]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              44677609  
                                                                 
 dense (Dense)               (None, 64)                64128     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
Total params: 44742777 (170.68 MB)
Trainable params: 65168 (254.56 KB)
Non-trainable params: 44677609 (170.43 MB)
_________________________________________________________________


In [4]:
from pathlib import Path
path = Path.cwd()/ Path('chest_xray')
list_folders = [p for p in path.rglob('*') if p.is_dir()]
list_folders = list_folders[2:]
list_folders
test_normal = [p for p in list_folders[0].rglob('*') if p.is_file()]
test_pneumonia = [p for p in list_folders[1].rglob('*') if p.is_file()]
train_normal = [p for p in list_folders[2].rglob('*') if p.is_file()]
train_pneumonia = [p for p in list_folders[3].rglob('*') if p.is_file()]
print('test files normal/pneumonia: ', len(test_normal) , '/' , len(test_pneumonia))
print('train files normal/pneumonia: ', len(train_normal) , '/' , len(train_pneumonia))


test files normal/pneumonia:  234 / 390
train files normal/pneumonia:  1341 / 3875


In [5]:
def file_path_to_image_pil(path):
    return Image.open(str(path))

def file_path_to_image_numpy(path):
    return np.array(Image.open(str(path)))

def image_numpy_to_image_numpy_resize(image_np):
    return cv2.resize(image_np,(224,224))

def image_numpy_to_image_pil(image_np):
    return Image.fromarray(image_np)
    image1 = np.array(Image.open(str(image)))
    resized_img = cv2.resize(image1,(224,224))
    
X_test, y_test, X_train, y_train = [],[],[],[]

def file_path_to_img_normalization(path):
    image = file_path_to_image_numpy(path)
    image = image_numpy_to_image_numpy_resize(image)
    return image
    
for file_path in test_normal:
    X_test.append(file_path_to_img_normalization(file_path))
    y_test.append(0)

for file_path in test_pneumonia:
    X_test.append(file_path_to_img_normalization(file_path))
    y_test.append(1)
    
for file_path in train_normal:
    X_train.append(file_path_to_img_normalization(file_path))
    y_train.append(0)
    
for file_path in train_pneumonia:
    X_train.append(file_path_to_img_normalization(file_path))
    y_train.append(1) 

In [6]:
RGB = 0
GREY = 0
Error = 0
for group in [X_test, X_train]:
    for image in group:
        if image.shape == (224, 224, 3):
            RGB+=1
        elif image.shape == (224, 224):
            GREY += 1
        else:
            print ("Error")
print("RGB=", RGB , "; GREY=",GREY, "; Error=",Error)

RGB= 283 ; GREY= 5557 ; Error= 0


In [7]:
for i in range(len(X_test)):
    if X_test[i].shape == (224, 224):
        X_test[i] = np.repeat(X_test[i][:, :, np.newaxis], 3, axis=2)
        
for i in range(len(X_train)):
    if X_train[i].shape == (224, 224):
        X_train[i] = np.repeat(X_train[i][:, :, np.newaxis], 3, axis=2)

In [8]:
temp1 = X_test[0][:,:,0]
temp2 = X_test[0][:,:,1]
temp3 = X_test[0][:,:,2]
if (np.array_equal(temp1,temp2)) and (np.array_equal(temp1,temp3)):
    print("OK")
print(temp1,"\n",30*"-","\n",temp2,"\n",30*"-","\n",temp3)

OK
[[32 32 32 ... 32 30 35]
 [32 32 29 ... 30 33 33]
 [32 32 34 ... 32 34 31]
 ...
 [32 30 28 ... 33 33 32]
 [29 27 29 ... 32 33 35]
 [31 31 29 ... 33 31 34]] 
 ------------------------------ 
 [[32 32 32 ... 32 30 35]
 [32 32 29 ... 30 33 33]
 [32 32 34 ... 32 34 31]
 ...
 [32 30 28 ... 33 33 32]
 [29 27 29 ... 32 33 35]
 [31 31 29 ... 33 31 34]] 
 ------------------------------ 
 [[32 32 32 ... 32 30 35]
 [32 32 29 ... 30 33 33]
 [32 32 34 ... 32 34 31]
 ...
 [32 30 28 ... 33 33 32]
 [29 27 29 ... 32 33 35]
 [31 31 29 ... 33 31 34]]


In [9]:
def normalization(image_np):
    return image_np/255

def undo_normalization():
    return (image_np*255).astype(np.uint8)

X_test_np = np.array(X_test)
X_train_np = np.array(X_train)

y_test_np = np.array(y_test)
y_train_np = np.array(y_train)

X_test_scaled = normalization(X_test_np)
X_train_scaled = normalization(X_train_np)

In [10]:
X_train = model.predict(X_train_scaled)
X_test = model.predict(X_test_scaled)

20/20 [==============================] - 16s 745ms/step


In [11]:
X_train.shape

(5216, 16)

In [12]:
y_train_np.shape

(5216,)

In [13]:
X_test.shape

(624, 16)

In [14]:
y_test_np.shape

(624,)

In [15]:
!pip install pycaret

In [16]:
X = np.vstack((X_train, X_test))
X.shape

(5840, 16)

In [17]:
y = np.concatenate((y_train_np, y_test_np))
y.shape

(5840,)

In [18]:
data_full = np.hstack((X, y.reshape(-1, 1)))
data_full.shape


(5840, 17)

In [19]:
import pandas as pd

df = pd.DataFrame(data_full)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,1.158536,0.000000,0.0,0.407447,0.000000,0.0,0.000000,0.000000,0.783899,1.717193,0.000000,0.000000,2.060250,0.481997,0.0,0.0
1,0.000000,1.584510,0.000000,0.0,1.718683,0.000000,0.0,0.000000,0.000000,1.244361,0.814854,0.000000,0.000000,2.230511,0.085195,0.0,0.0
2,0.000000,2.761390,0.000000,0.0,1.975990,0.000000,0.0,0.000000,0.000000,2.168732,1.976272,0.000000,0.000000,5.739696,0.054311,0.0,0.0
3,0.000000,1.926991,0.000000,0.0,1.026134,0.000000,0.0,0.000000,0.000000,1.718145,1.486154,0.000000,0.000000,4.407917,0.000000,0.0,0.0
4,0.000000,0.894852,0.000000,0.0,1.446362,0.000000,0.0,0.000000,0.000000,1.029538,1.307556,0.000000,0.000000,2.864645,1.198703,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5835,2.589634,0.000000,6.876414,0.0,0.000000,8.591226,0.0,2.174064,5.135255,0.000000,0.000000,9.912086,3.272862,0.000000,10.029964,0.0,1.0
5836,2.746883,0.000000,6.964716,0.0,0.000000,4.931220,0.0,1.817608,2.229113,0.000000,0.000000,7.554579,4.087494,0.000000,8.482302,0.0,1.0
5837,1.997853,0.000000,2.427520,0.0,0.000000,8.455653,0.0,1.601164,5.687395,0.000000,0.000000,9.981335,1.989179,0.000000,7.536919,0.0,1.0
5838,2.094653,0.000000,2.627221,0.0,0.000000,2.975150,0.0,0.680309,2.050772,0.000000,0.000000,5.032138,3.099306,0.000000,4.069820,0.0,1.0


In [20]:
data[0]

NameError: name 'data' is not defined

In [21]:
df.iloc[:,-1]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
5835    1.0
5836    1.0
5837    1.0
5838    1.0
5839    1.0
Name: 16, Length: 5840, dtype: float64

In [ ]:
'''%%time
from pycaret.classification import *
pycaret_classification = setup(data = data_full, target = '16')'''

In [ ]:
'''%%time
from pycaret.classification import *
pycaret_classification = setup(data = df, target = df.iloc[:,-1])'''

In [22]:
import numpy as np
import pandas as pd

In [24]:
df.shape

(5840, 17)

In [49]:
df_train = df.iloc[0:4672,:-1]
df_train.shape

(4672, 16)

In [50]:
df_test = df.iloc[-1168:,:-1]
df_test.shape

(1168, 16)

In [51]:
df_train_y = df.iloc[0:4672,-1:]
df_train_y.shape

(4672, 1)

In [52]:
df_test_y = df.iloc[-1168:,-1:]
df_test_y.shape

(1168, 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test, df_train_y, df_test_y = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [53]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [54]:
from sklearn.tree import ExtraTreeClassifier 


In [55]:
parameters = {'criterion': 'gini', 'max_depth': 25, 'min_samples_leaf': 6, 'splitter': 'best'}

classifier = ExtraTreeClassifier(**parameters, random_state = 0)

In [56]:
classifier.fit(X_train, y_train)

ExtraTreeClassifier(max_depth=25, min_samples_leaf=6, random_state=0,
                    splitter='best')

In [58]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = classifier, X = df_train, y = df_train_y, cv = 10)

print("Mean accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))
print("Accuracy of each split: " , accuracies)

Mean accuracy: 96.90 %
Standard Deviation: 1.08 %
Accuracy of each split:  [0.97435897 0.95726496 0.97858672 0.9764454  0.98286938 0.94646681
 0.97430407 0.96573876 0.97430407 0.95931478]


In [60]:
prediction = classifier.predict(df_test)
print("Matrix prediction for X_test: ",prediction)

Matrix prediction for X_test:  [1 1 1 ... 1 1 1]


In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(df_test_y, prediction)
print('Confusion matrix: ')
print(cm)

print(f'True Positive:{cm[1][1]}  True Negative:{cm[0][0]}  False Positive:{cm[0][1]}  False Negative:{cm[1][0]}')

right = cm[1][1]+cm[0][0]
wrong = cm[1][0]+cm[0][1]
print(f'Predict right: {right} - Predict wrong: {wrong}')

acc = accuracy_score(df_test_y, prediction)
print('Accurate: ',acc)


Confusion matrix: 
[[135  99]
 [ 18 916]]
True Positive:916  True Negative:135  False Positive:99  False Negative:18
Predict right: 1051 - Predict wrong: 117
Accurate:  0.8998287671232876


In [ ]:
%%time
from pycaret.classification import *
pycaret_classification = setup(data = df, target = "16")

In [ ]:
best_model = compare_models(fold=10)

In [ ]:
et = create_model('et')

In [ ]:
plot_model(et, plot = 'confusion_matrix')

# 422+51+22+1247 = 1753 Transformed test set shape	(1753, 17)

In [ ]:
save_model(et,"Final Model v100 ")

In [ ]:
evaluate_model(et)